In [68]:

# ========== ----- ========== Import Libraries ========== ----- ========== #

import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
import torch
import torchvision.models as models
import sys
import os
import cv2
import matplotlib.pyplot as plt

# ========== ----- ========== End ========== ----- ========== #

In [69]:
# ========== ----- ========== Resize Images ========== ----- ========== #

# If you want to resize all the images in the list to the same size, it's usually a good idea to choose a target size
# that is smaller than the largest image in the list, but larger than the smallest image. This way, you can ensure that
# all the images will be scaled down without losing too much detail or becoming too small.
# For example, if the largest image has a size of(3072, 4592, 3) and the smallest image has a size of(155, 155, 3),
# you could choose a target size of(512, 512, 3). This size should be large enough to capture most of the details in
# the larger images while still being a significant improvement over the smaller images.

# Set the path to your input and output directories
input_dirs = ['dataset/flooded', 'dataset/non-flooded']

img_sizes = []
for input_dir in input_dirs:
    # Loop through all the files in the input directory
    for filename in os.listdir(input_dir):
        # Read the image from the input directory
        img = cv2.imread(os.path.join(input_dir, filename))
        img_sizes.append(img.shape)



In [70]:
print("Minimum size of flooded & non-flooded images: ", min(img_sizes))
print("Maximuim size of flooded & non-flooded images: ", max(img_sizes),"\n")
print("Target size of flooded & non-flooded images will be ", (512, 512, 3))

# ========== ----- ========== End ========== ----- ========== #

Minimum size of flooded & non-flooded images:  (152, 123, 3)
Maximuim size of flooded & non-flooded images:  (3072, 4592, 3) 

Target size of flooded & non-flooded images will be  (512, 512, 3)


In [71]:
# ========== ----- ========== Haze Removal Function ========== ----- ========== #

def haze_Removal(img):
    # The same pixels threshold for all (their histograms are alike)
    threshold = 100

    # Copy original image into a new image.
    new_img = img.copy()

    # Means we will shift by the first grey-level where there is less than 500 pixels
    # This was set by judging from results and image size

    for band_num in range(img.shape[2]):
        # notice that min, max are called argmin and argmax here (since that's what they really are.)

        img_band = img[:, :, band_num]
        hist = cv2.calcHist([img_band], [0], None, [256], [0, 256])

        # The first index (BV) where there is atleast "threshold" no. of pixels
        argmin = np.where(hist > threshold)[0][0]

        new_img_band = new_img[:, :, band_num]
        
        # To avoid shifting beyond zero
        big_vals = new_img_band > argmin
        new_img_band[big_vals] = new_img_band[big_vals] - argmin

        new_img[:, :, band_num] = new_img_band
    
    return new_img

# ========== ----- ========== End ========== ----- ========== #


In [72]:
# ========== ----- ========== Preprocessing Function ========== ----- ========== #

# Set the path to your input and output directories
output_dirs = ['dataset_preprocessed/flooded',
               'dataset_preprocessed/non-flooded'
               ]
new_size = (512, 512)

# Loop through all the files in the input directory
for input_dir in input_dirs:
    for filename in os.listdir(input_dir):

        # Read the image from the input directory
        img = cv2.imread(os.path.join(input_dir, filename))

        # Resize the image to a desired size (e.g. 512*512)
        img = cv2.resize(img, new_size)

        # # Use bicubic interpolation to enhance the image resolution
        img = cv2.resize(img, None, fx=1, fy=1, interpolation=cv2.INTER_CUBIC)

        # Apply Haze Removal by Dark Subtraction
        img = haze_Removal(img)

        # Save the processed image to the output directory
        if (input_dir == 'dataset/flooded'):
            cv2.imwrite(os.path.join(output_dirs[0], filename), img)
        else:
            cv2.imwrite(os.path.join(output_dirs[1], filename), img)


# ========== ----- ========== End ========== ----- ========== #
